User Data: User information is stored in the /users/ Firestore collection. Each document in this collection represents a user and contains fields like name, email, current_package, and remaining_investigations.

Subscriptions: Each user has a /subscriptions/ subcollection. Each document in this subcollection represents a subscription the user has or had. It contains fields like package, start_date, end_date, payment_status, and payment_intent.

Payments: Payments are stored in the /payments/ Firestore collection. Each document in this collection represents a payment made by a user. It contains fields like user_id, subscription_id, amount, date, status, and payment_intent.

Investigations: Investigations are stored in the /investigations/ Firestore collection. Each document in this collection represents an investigation initiated by a user. It contains fields like user_id, timestamp, and asins.

Stripe Integration: When a user decides to subscribe to a package, a Stripe PaymentIntent is created. The client_secret of this PaymentIntent is sent to the front end, which confirms the payment with Stripe. Stripe then sends a webhook event to your server, which updates the payment_status of the subscription in Firestore and creates a new payment document in the /payments/ collection.

Investigations: Users spend their remaining investigations to initiate investigations. Each investigation can involve multiple ASINs. When a user initiates an investigation, a new document is created in the /investigations/ collection and the user's remaining investigations are decremented.

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('path/to/serviceAccount.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

def subscribe_user_to_package(user_id, package, start_date, payment_intent_id):
    user_ref = db.collection(u'users').document(user_id)
    subs = user_ref.collection(u'subscriptions').document()
    subs.set({
        'package': package,
        'start_date': start_date,
        'payment_status': 'pending',  # payment has not yet been confirmed
        'payment_intent': payment_intent_id,  # store the Stripe PaymentIntent ID
    })
    user_ref.update({
        'current_package': package,
        # reset remaining_investigation based on package
        'remaining_investigation': 50 if package == 'basic' else 100,  # assuming premium users get 100 analyses
    })

def log_payment(user_id, subscription_id, amount, date, status, payment_intent_id):
    payments_ref = db.collection(u'payments').document()
    payments_ref.set({
        'user_id': user_id,
        'subscription_id': subscription_id,
        'amount': amount,
        'date': date,
        'status': status,
        'payment_intent': payment_intent_id,  # store the Stripe PaymentIntent ID
    })

def use_investigation(user_id):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_investigation')
    if remaining > 0:
        user_ref.update({
            'remaining_investigation': firestore.Increment(-1),
        })
        return True  # investigation was successful
    else:
        return False  # user is out of investigation


In [ ]:
def add_investigation(user_id, asins):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_investigations')
    if remaining > 0:
        investigation_ref = db.collection(u'investigations').document()
        investigation_ref.set({
            'user_id': user_id,
            'received_timestamp': firestore.SERVER_TIMESTAMP,
            'asins': asins,
            'status': 'received',
            'started_timestamp': None,
            'finished_timestamp': None,
            'reviewed_timestamps': [],
        })
        user_ref.update({
            'remaining_investigations': firestore.Increment(-1),
        })
        return True  # investigation was successful
    else:
        return False  # user is out of investigations

def update_investigation_status(investigation_id, new_status):
    investigation_ref = db.collection(u'investigations').document(investigation_id)
    investigation = investigation_ref.get()
    if investigation.exists:
        investigation_ref.update({
            'status': new_status,
            f'{new_status}_timestamp': firestore.SERVER_TIMESTAMP,
        })
        return True  # update was successful
    else:
        return False  # investigation does not exist

def log_investigation_review(investigation_id):
    investigation_ref = db.collection(u'investigations').document(investigation_id)
    investigation = investigation_ref.get()
    if investigation.exists and investigation.get('status') == 'finished':
        investigation_ref.update({
            'reviewed_timestamps': firestore.ArrayUnion([firestore.SERVER_TIMESTAMP]),
        })
        return True  # log was successful
    else:
        return False  # investigation does not exist or is not finished yet

def has_investigations_available(user_id):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_investigations')
    return remaining > 0

# Rest of your code remains same


In [ ]:
user_id = '...'
if has_investigations_available(user_id):
    asins = ['B0BCTTCSBZ', 'B0BLNBS36G', 'B0BW8Y2B8Z', 'B091325ZMB']
    add_investigation(user_id, asins)
else:
    print('User has no remaining investigations')


In [ ]:
import stripe

stripe.api_key = 'your_stripe_api_key'

def create_payment(user_id, package, amount):
    payment_intent = stripe.PaymentIntent.create(
        amount=amount,  # amount in cents
        currency='usd',
        metadata={'user_id': user_id, 'package': package},
    )
    return payment_intent.client_secret


In [ ]:
from flask import Flask, request
import stripe

app = Flask(__name__)

stripe.api_key = 'your_stripe_api_key'

@app.route('/webhook', methods=['POST'])
def webhook_received():
    event = stripe.Webhook.construct_event(
        request.data, request.headers.get('stripe-signature'), 'your_stripe_webhook_secret'
    )

    if event['type'] == 'payment_intent.succeeded':
        payment_intent = event['data']['object']
        user_id = payment_intent['metadata']['user_id']
        package = payment_intent['metadata']['package']
        log_payment(user_id, package, payment_intent['amount'], payment_intent['created'], 'completed')

    return '', 200
